In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')



In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./Defense_Distillation.csv", mode='a', index=False)




In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
epochs = 50
training_batch_size = 128
validation_batch_size = 128
learning_rate = 0.01
baseline_temperature = 1
distilled_temperature = 100


x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)


input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize models
teacher_model = DNNModel(input_shape, output_shape).to(device)
student_model = DNNModel(input_shape, output_shape).to(device)


In [5]:
# Training function for the teacher model
def train_teacher_model(model, train_loader, epochs, temperature):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_loss = 0.0
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs / temperature, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(model.state_dict(), 'teacher_model.pth')



In [6]:
# Train the teacher model
# train_teacher_model(teacher_model, train_loader, epochs, baseline_temperature)

In [7]:
teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/Defense_Distillation/teacher_model.pth"))

/tmp/ipykernel_10912/2830116531.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL

<All keys matched successfully>

In [8]:
# Distillation loss function
def distillation_loss(student_outputs, teacher_outputs, temperature, alpha):
    soft_targets = F.softmax(teacher_outputs / temperature, dim=1)
    student_log_probs = F.log_softmax(student_outputs / temperature, dim=1)
    distillation_loss = nn.KLDivLoss()(student_log_probs, soft_targets) * (temperature ** 2)
    return distillation_loss

# Training function for the student model using distillation
def train_student_model(student_model, teacher_model, train_loader, epochs, temperature, alpha=0.5):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')
    
    student_model.train()
    teacher_model.eval()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    
    train_loss = 0.0

    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            teacher_outputs = teacher_model(inputs).detach()
            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        student_model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                teacher_outputs = teacher_model(inputs).detach()
                student_outputs = student_model(inputs)
                loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
                val_train_loss += loss.item()
                _, predicted = torch.max(student_outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(student_model.state_dict(), 'student_model.pth')



In [9]:
# Train the student model
# train_student_model(student_model, teacher_model, train_loader, epochs, distilled_temperature)


In [10]:
student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/Defense_Distillation/student_model.pth"))

/tmp/ipykernel_10912/1967020892.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL

<All keys matched successfully>

In [11]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)




In [12]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def8/y_pred_{attack_name}{eps}_Def8.npy", all_preds)


In [13]:

# calculate_performance_metrics(x_test, y_test, student_model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)

In [15]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(228, 42) (228,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(217675, 42) (217675,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9998346158263466

macro
Precision: 0.6798780487804879
Recall: 0.7455223880597015
F1 Score: 0.70696082201145

weighted
Precision: 0.9999239748226126
Recall: 0.9998346158263466
F1 Score: 0.9998723474717924

start XGB 100 FGSM
(218988, 42) (218988,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9973423201271302

macro
Precision: 0.5326512330467068
Recall: 0.703706649697741
F1 Score: 0.5322837518386928

weighted
Precision: 0.9995710705249972
Recall: 0.9973423201271302
F1 Score: 0.9982036177972208

start XGB 100 PGD
(217675, 42) (217675,)


...

Accuracy: 0.9975961538461539

macro
Precision: 0.6666666666666666
Recall: 0.64
F1 Score: 0.6527777777777778

weighted
Precision: 1.0
Recall: 0.9975961538461539
F1 Score: 0.9987479967948718

start RF 1 BIM
(16467, 42) (16467,)
Accuracy: 0.9911337827169491

macro
Precision: 0.7789194153382673
Recall: 0.8122308862725959
F1 Score: 0.7775481455208194

weighted
Precision: 0.9928026601019975
Recall: 0.9911337827169491
F1 Score: 0.9917141366314876

start RF 1 FGSM
(16749, 42) (16749,)
Accuracy: 0.9879992835393158

macro
Precision: 0.7211563011655774
Recall: 0.7782789494583072
F1 Score: 0.7414928140000827

weighted
Precision: 0.9892715482370286
Recall: 0.9879992835393158
F1 Score: 0.9883355257239846

start RF 1 PGD
(16467, 42) (16467,)
Accuracy: 0.9911337827169491

macro
Precision: 0.7789194153382673
Recall: 0.8122308862725959
F1 Score: 0.7775481455208194

weighted
Precision: 0.9928026601019975
Recall: 0.9911337827169491
F1 Score: 0.9917141366314876

start RF 1 DF
(3901, 42) (3901,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.528069725711356

macro
Precision: 0.19463834993259335
Recall: 0.12254441104126812
F1 Score: 0.14297602295301876

weighted
Precision: 0.9118416166644426
Recall: 0.528069725711356
F1 Score: 0.6686167804831319

start RF 1 AutoPGD
(59193, 42) (59193,)
Accuracy: 0.9996283344314362

macro
Precision: 0.94741943712175
Recall: 0.9346532118772697
F1 Score: 0.9393833362959578

weighted
Precision: 0.9996118618657727
Recall: 0.9996283344314362
F1 Score: 0.999614285285328

start RF 1 ZOO
(2531, 42) (2531,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9893322797313315

macro
Precision: 0.5625
Recall: 0.6989247311827957
F1 Score: 0.5715568862275449

weighted
Precision: 0.9941722639273015
Recall: 0.9893322797313315
F1 Score: 0.9905251527762333

start RF 1 CaFA
(640329, 42) (640329,)
Accuracy: 0.6250568067352876

macro
Precision: 0.3207357953022136
Recall: 0.3349271341507342
F1 Score: 0.279323523356625

weighted
Precision: 0.9485368123694836
Recall: 0.6250568067352876
F1 Score: 0.7517849154390037

start RF 1 SINIFGSM
(17360, 42) (17360,)
Accuracy: 0.9952764976958526

macro
Precision: 0.842899642820025
Recall: 0.8812160470399928
F1 Score: 0.8469596013251877

weighted
Precision: 0.9960761888386707
Recall: 0.9952764976958526
F1 Score: 0.9954790146695623

start RF 1 VNIFGSM
(17227, 42) (17227,)
Accuracy: 0.9928600452777616

macro
Precision: 0.8622745744084591
Recall: 0.8903310729894487
F1 Score: 0.8735596863734971

weighted
Precision: 0.9935090384085933
Recall: 0.9928600452777616
F1 Score: 0.9930848043938312



In [16]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_baseline_Def8.npy not found
start XGB 100 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_BIM_Def8.npy not found
start XGB 100 FGSM
(135, 42) (135,)
Accuracy: 0.014814814814814815

macro
Precision: 0.25
Recall: 0.003703703703703704
F1 Score: 0.0072992700729927005

weighted
Precision: 1.0
Recall: 0.014814814814814815
F1 Score: 0.029197080291970802

start XGB 100 PGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_PGD_Def8.npy not found
start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(3142, 42) (3142,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.6499045194143858

macro
Precision: 0.2502302025782689
Recall: 0.41244826488379494
F1 Score: 0.1973916682082234

weighted
Precision: 0.9996820244463122
Recall: 0.6499045194143858
F1 Score: 0.7874766233172742

start XGB 100 AutoPGD
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_ZOO_Def8.npy not found
start XGB 100 CaFA
(5, 42) (5,)
Accuracy: 0.8

macro
Precision: 0.5555555555555555
Recall: 0.6666666666666666
F1 Score: 0.6

weighted
Precision: 0.6666666666666666
Recall: 0.8
F1 Score: 0.72

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_SINIFGSM_Def8.npy not found
start XGB 100 VNIFGSM
(1, 42) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0

...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6062689029419851

macro
Precision: 0.25
Recall: 0.15156722573549627
F1 Score: 0.18871961656966793

weighted
Precision: 1.0
Recall: 0.6062689029419851
F1 Score: 0.7548784662786717

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_AutoPGD_Def8.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_ZOO_Def8.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_CaFA_Def8.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_SINIFGSM_Def8.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_VNIFGSM_Def8.npy not found
start RF 10 baseline
/hom

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6042780748663101

macro
Precision: 0.25
Recall: 0.15106951871657753
F1 Score: 0.18833333333333332

weighted
Precision: 1.0
Recall: 0.6042780748663101
F1 Score: 0.7533333333333333

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_AutoPGD_Def8.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_ZOO_Def8.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_CaFA_Def8.npy not found
start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_SINIFGSM_Def8.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_VNIFGSM_Def8.npy not found
start RF 1 baseline
/home

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_baseline_Def8.npy not found
start XGB 100 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_BIM_Def8.npy not found
start XGB 100 FGSM
(519, 42) (519,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start XGB 100 PGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_PGD_Def8.npy not found
start XGB 100 DF
(3241, 42) (3241,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.6729404504782475

macro
Precision: 0.2
Recall: 0.13471278567016678
F1 Score: 0.16098911238235838

weighted
Precision: 0.9990743597655045
Recall: 0.6729404504782475
F1 Score: 0.8042004719131078

start XGB 100 AutoPGD
(41, 42) (41,)
Accuracy: 0.6829268292682927

macro
Precision: 0.5
Recall: 0.34146341463414637
F1 Score: 0.4057971014492754

weighted
Precision: 1.0
Recall: 0.6829268292682927
F1 Score: 0.8115942028985507

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_ZOO_Def8.npy not found
start XGB 100 CaFA
(2, 42) (2,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_SINIFGSM_Def8.npy not found
start XGB 100 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/X

...

Accuracy: 0.5798368298368298

macro
Precision: 0.25
Recall: 0.14495920745920746
F1 Score: 0.183511619328661

weighted
Precision: 1.0
Recall: 0.5798368298368298
F1 Score: 0.734046477314644

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_AutoPGD_Def8.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_ZOO_Def8.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_CaFA_Def8.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_SINIFGSM_Def8.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_VNIFGSM_Def8.npy not found
start RF 10 b

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6009933774834437

macro
Precision: 0.25
Recall: 0.15024834437086093
F1 Score: 0.187693898655636

weighted
Precision: 1.0
Recall: 0.6009933774834437
F1 Score: 0.750775594622544

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_AutoPGD_Def8.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_ZOO_Def8.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_CaFA_Def8.npy not found
start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_SINIFGSM_Def8.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_VNIFGSM_Def8.npy not found
start RF 1 ba

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5368642090527298

macro
Precision: 0.2
Recall: 0.10737284181054596
F1 Score: 0.13972977076058904

weighted
Precision: 1.0
Recall: 0.5368642090527298
F1 Score: 0.6986488538029452

start RF 1 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_AutoPGD_Def8.npy not found
start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_ZOO_Def8.npy not found
start RF 1 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_CaFA_Def8.npy not found
start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_SINIFGSM_Def8.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_VNIFGSM_Def8.npy not found
start XGB 100 baselin

In [15]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start BIM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8993928881179531

macro
Precision: 0.6231974921630093
Recall: 0.7470453155191331
F1 Score: 0.6671430441518104

weighted
Precision: 0.9072217222619471
Recall: 0.8993928881179531
F1 Score: 0.8956557042618778

start FGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.36032093362509116

macro
Precision: 0.2886865148861646
Recall: 0.44072484501669057
F1 Score: 0.21264689032238865

weighted
Precision: 0.8144961748298825
Recall: 0.36032093362509116
F1 Score: 0.33876434971122504

start PGD
Accuracy: 0.8993928881179531

macro
Precision: 0.6231974921630093
Recall: 0.7470453155191331
F1 Score: 0.6671430441518104

weighted
Precision: 0.9072217222619471
Recall: 0.8993928881179531
F1 Score: 0.8956557042618778



...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9903466235632183

macro
Precision: 0.502217137555991
Recall: 0.48541529142235856
F1 Score: 0.4820377253703258

weighted
Precision: 0.9838540669785842
Recall: 0.9903466235632183
F1 Score: 0.987055025138014

start ZOO
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start CaFA
Accuracy: 0.6100299285127235

macro
Precision: 0.32932061330094287
Recall: 0.44242180288038657
F1 Score: 0.2940132484658665

weighted
Precision: 0.9715646665216218
Recall: 0.6100299285127235
F1 Score: 0.7463593162318869

start SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.30129179331306993

macro
Precision: 0.5224165778167192
Recall: 0.5070201256131665
F1 Score: 0.3546172658927542

weighted
Precision: 0.8575390240552149
Recall: 0.30129179331306993
F1 Score: 0.3027632030919775

start VNIFGSM
Accuracy: 0.8526748971193415

macro
Precision: 0.5474262086523567
Recall: 0.703052778052778
F1 Score: 0.5569747815661792

weighted
Precision: 0.8887698600971286
Recall: 0.8526748971193415
F1 Score: 0.8590967939277627



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(228, 42) (228,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(217675, 42) (217675,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9998346158263466

macro
Precision: 0.6798780487804879
Recall: 0.7455223880597015
F1 Score: 0.70696082201145

weighted
Precision: 0.9999239748226126
Recall: 0.9998346158263466
F1 Score: 0.9998723474717924

start XGB 100 FGSM
(218988, 42) (218988,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9973423201271302

macro
Precision: 0.5326512330467068
Recall: 0.703706649697741
F1 Score: 0.5322837518386928

weighted
Precision: 0.9995710705249972
Recall: 0.9973423201271302
F1 Score: 0.9982036177972208

start XGB 100 PGD
(217675, 42) (217675,)


...

(221241, 42) (221241,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.9886458658205306

macro
Precision: 0.2681589218310215
Recall: 0.22000207440171787
F1 Score: 0.2321205608697595

weighted
Precision: 0.9972637063042157
Recall: 0.9886458658205306
F1 Score: 0.9928441319420576

start RF 10 SINIFGSM
(3, 42) (3,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_VNIFGSM_Def8.npy not found
start RF 1 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_baseline_Def8.npy not found
start RF 1 BIM
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 FGSM
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 PGD
(1, 42) (1,)
Accuracy:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9861883533749639

macro
Precision: 0.2295071382617206
Recall: 0.24132831696618498
F1 Score: 0.2340152213388484

weighted
Precision: 0.998676332140413
Recall: 0.9861883533749639
F1 Score: 0.9923900966120316

start RF 1 SINIFGSM
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 VNIFGSM
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



In [ ]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(228, 42) (228,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(217675, 42) (217675,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9998346158263466

macro
Precision: 0.6798780487804879
Recall: 0.7455223880597015
F1 Score: 0.70696082201145

weighted
Precision: 0.9999239748226126
Recall: 0.9998346158263466
F1 Score: 0.9998723474717924

start XGB 100 FGSM
(218988, 42) (218988,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9973423201271302

macro
Precision: 0.5326512330467068
Recall: 0.703706649697741
F1 Score: 0.5322837518386928

weighted
Precision: 0.9995710705249972
Recall: 0.9973423201271302
F1 Score: 0.9982036177972208

start XGB 100 PGD
(217675, 42) (217675,)


...

Accuracy: 0.989704249520541

macro
Precision: 0.3333333333333333
Recall: 0.329901416506847
F1 Score: 0.33160849567099565

weighted
Precision: 1.0
Recall: 0.989704249520541
F1 Score: 0.994825487012987

start RF 1 FGSM
(9761, 42) (9761,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9931359491855343

macro
Precision: 0.3333333333333333
Recall: 0.3310453163951781
F1 Score: 0.33218538507667267

weighted
Precision: 1.0
Recall: 0.9931359491855343
F1 Score: 0.9965561552300181

start RF 1 PGD
(9907, 42) (9907,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.989704249520541

macro
Precision: 0.3333333333333333
Recall: 0.329901416506847
F1 Score: 0.33160849567099565

weighted
Precision: 1.0
Recall: 0.989704249520541
F1 Score: 0.994825487012987

start RF 1 DF
(2, 42) (2,)
Accuracy: 0.5

macro
Precision: 0.5
Recall: 0.25
F1 Score: 0.3333333333333333

weighted
Precision: 1.0
Recall: 0.5
F1 Score: 0.6666666666666666

start RF 1 AutoPGD
(13175, 42) (13175,)
Accuracy: 0.9996963946869071

macro
Precision: 0.8666666666666666
Recall: 0.9999392281981161
F1 Score: 0.8999696094818417

weighted
Precision: 0.999898798228969
Recall: 0.9996963946869071
F1 Score: 0.9997722729483732

start RF 1 ZOO
(45177, 42) (45177,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 CaFA
(496848, 42) (496848,)


In [ ]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



In [15]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9998346158263466

macro
Precision: 0.6798780487804879
Recall: 0.7455223880597015
F1 Score: 0.70696082201145

weighted
Precision: 0.9999239748226126
Recall: 0.9998346158263466
F1 Score: 0.9998723474717924

start XGB 100 FGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9973423201271302

macro
Precision: 0.5326512330467068
Recall: 0.703706649697741
F1 Score: 0.5322837518386928

weighted
Precision: 0.9995710705249972
Recall: 0.9973423201271302
F1 Score: 0.9982036177972208

start XGB 100 PGD


...

Accuracy: 0.6420198376916141

macro
Precision: 0.2502278942570647
Recall: 0.2855263157894737
F1 Score: 0.19596777956650568

weighted
Precision: 0.9993994058172698
Recall: 0.6420198376916141
F1 Score: 0.7815822520267335

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input20/x_test_adv_AutoPGD_Def8.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input20/x_test_adv_ZOO_Def8.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input20/x_test_adv_CaFA_Def8.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input20/x_test_adv_SINIFGSM_Def8.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WU

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.6956661316211878

macro
Precision: 0.2502866972477064
Recall: 0.29894796016704145
F1 Score: 0.2057245733819464

weighted
Precision: 0.9993586817264789
Recall: 0.6956661316211878
F1 Score: 0.8200845568593853

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_AutoPGD_Def8.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_ZOO_Def8.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_CaFA_Def8.npy not found
start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input10/x_test_adv_SINIFGSM_Def8.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WU

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6381653454133636

macro
Precision: 0.25021150592216584
Recall: 0.28456090651558075
F1 Score: 0.19521738635142144

weighted
Precision: 0.9994342276464828
Recall: 0.6381653454133636
F1 Score: 0.7787401007425288

start RF 1 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input1/x_test_adv_AutoPGD_Def8.npy not found
start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input1/x_test_adv_ZOO_Def8.npy not found
start RF 1 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input1/x_test_adv_CaFA_Def8.npy not found
start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude2Attack/WUSTL_Input1/x_test_adv_SINIFGSM_Def8.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dyn

In [15]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student_model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")